In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score
from sklearn import linear_model
from sklearn import svm

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.utils.np_utils import to_categorical

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
# from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df_orig = pd.read_csv('/kaggle/input/train_2kmZucJ.csv')
test_df_orig = pd.read_csv('/kaggle/input/test_oJQbWVk.csv')
submit = pd.read_csv('/kaggle/input/sample_submission_LnhVWA4.csv')

train_df = train_df_orig.copy()
test_df = test_df_orig.copy()

In [ ]:
train_df.head(10)

In [ ]:
train_df.tweet[0]

Our data contains:
1. hashtags
2. links (urls)
3. interrogative and exclamative sentences

What we want to do?:
1. remove links
2. hashtags are helpful so keep them but remove '#'
3. map(replace) emojis with words

Text Preprocessing:
1. text lowering
* remove noise
* remove stop words
* lemmatization and stemming
* normalization (hacker, hack3r ==> hacker)

In [ ]:
X = train_df['tweet']
y = train_df['label']

Reference for emoji meanings

ref: https://www.catholicidaho.org/media/1/52/SE%20Texting%20Symbols.pdf

In [ ]:
re.match?

In [ ]:
t = X[1]
print(t)
print(re.findall('\^\^', t))
print(re.findall(':\)', t))
print(re.findall('.*?', t))

In [ ]:
for i in range(0,10):
    print(X[i])
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    review = X[i]
    review = review.replace('$&@*#', 'vulgar')
    review = review.replace(';)', 'smile').replace(':)', 'smile').replace('^^', 'eyes').replace('(:', 'smile').replace('(;', 'smile')
#     review = review.replace('won\'t', 'will not').replace('shouldn\'t', 'should not')
    review = re.sub(regex, '', review)
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower().split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    print(review)
    print(y[i],end='\n')

look at i=1,3,4 values. They have to be fixed

In [ ]:
corpus = []
for i in range(0, len(X)):
    review = X[i]
#     regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
#     review = re.sub(regex, '', review)
    review = review.replace(';)', 'smile').replace(':)', 'smile').replace('^^', 'eyes').replace('(:', 'smile').replace('(;', 'smile').replace('$&@*#', 'vulgar')
    review = re.sub('[^a-zA-Z]', ' ', review) # removing noice using regex
    review = review.lower() # case lowering of string sentence
    review = review.split() # splitting sentence by space to get a list of words
    ps = PorterStemmer() # to stem words(removing ing,er..etc from words)
#     word_net_lemmatizer = WordNetLemmatizer() # not using it
    # stemming words which are not in stopwords
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))] 
    review = ' '.join(review) # rejoining words to form sentence
    corpus.append(review)
X = corpus

In [ ]:
test = []
for i in range(0, len(test_df.tweet)):
    review = test_df.tweet[i]
#     regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
#     review = re.sub(regex, '', review)
    review = review.replace(';)', 'smile').replace(':)', 'smile').replace('^^', 'eyes').replace('(:', 'smile').replace('(;', 'smile').replace('$&@*#', 'vulgar')
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    test.append(review)

In [ ]:
X[:10]

In [ ]:
countVectorizer = CountVectorizer(max_features=30000,ngram_range=(1, 3))
tfidf = TfidfVectorizer()

In [ ]:
train_vectors = countVectorizer.fit_transform(X)

## note that we're NOT using .fit_transform() here. Using just .transform() makes sure
# that the tokens in the train vectors are the only ones mapped to the test vectors - 
# i.e. that the train and test vectors use the same set of tokens.
test_vectors = countVectorizer.transform(test_df["tweet"])

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.2,random_state = 0)

In [ ]:
def preds_to_csv(preds, name):
    pd.DataFrame({"id":submit.id, "label":preds}).to_csv("sentiment_analysis_"+name+".csv", index=False)

In [ ]:
clf = linear_model.RidgeClassifier()

#try using countvectorizer in place of tfidf in below
pipeline = Pipeline([
        ('vectorizer', tfidf),
        ('classifier', clf)
    ])
model = pipeline.fit(train_X, train_y)
pred = model.predict(val_X)
test_pred = model.predict(test)
submit["ridge_classifier"] = test_pred
print(classification_report(val_y, pred))

In [ ]:
# # defining parameter range 
# param_grid = {'C': [0.1, 1, 10, 100],  
#               'gamma': [1, 0.1], 
#               'kernel': ['rbf']} 

# svc_model = GridSearchCV(svm.SVC(), param_grid, refit = True, verbose = 3)
pipeline = Pipeline([
        ('vectorizer', tfidf),
        ('classifier', svm.SVC(C=1, gamma=1)) #tuned parameters
    ])

model = pipeline.fit(train_X, train_y)
# tfidf.transform(val_X)
pred = model.predict(val_X)

test_pred = model.predict(test)
submit["svc"] = test_pred

print(classification_report(val_y, pred, target_names=['negative','positive']))

# print(svc_model.best_params_) 
# print(svc_model.best_estimator_)

In [ ]:
confusion_matrix(val_y, pred)

In [ ]:
confusion_matrix(val_y, pred)

In [ ]:
t = "apple is vulgar"
model.predict([t])

In [ ]:
preds_to_csv(preds = submit.svc, name="svc")

In [ ]:
for i in range(len(val_X)):
    # false positives
    if val_y.values[i]==0 and pred[i]==1:
        print(val_X[i])

In [ ]:
max_fatures = 30000
tokenizer = Tokenizer(nb_words=max_fatures, split=' ')
tokenizer.fit_on_texts(X)
X1 = tokenizer.texts_to_sequences(X)
X1 = pad_sequences(X1)

Y1 = pd.get_dummies(y).values
#Y = Y.reshape()
X1_train, X1_test, Y1_train, Y1_test = train_test_split(X1,Y1, test_size=0.2,random_state = 42)
print(X1_train.shape,Y1_train.shape)
print(X1_test.shape,Y1_test.shape)

embed_dim = 150
lstm_out = 100

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X1.shape[1]))
# model.add(Dropout(0.25))
model.add(LSTM(lstm_out))
# model.add(Dropout(0.25))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
# print(model.summary())

batch_size = 32
model.fit(X1_train, Y1_train, epochs = 10, batch_size=batch_size)

score,acc = model.evaluate(X1_test, Y1_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

In [ ]:
test_seq = pad_sequences(tokenizer.texts_to_sequences(test))
submit["lstm"] = np.argmax(model.predict(test_seq), axis=1)

In [ ]:
submit.head(2)

In [ ]:
parameters = {
    'num_class':2
}

train_tfidf_vectors = tfidf.fit_transform(train_X)
val_tfidf_vectors = tfidf.transform(val_X)
test_tfidf_vectors = tfidf.transform(test)

dtrain = xgb.DMatrix(train_tfidf_vectors, train_y)
dval = xgb.DMatrix(val_tfidf_vectors, val_y)

model = xgb.train(params = parameters,
                  dtrain = dtrain,
                  num_boost_round=1000,
                  evals = [(dval, "val")],
                  early_stopping_rounds = 20,
                  verbose_eval = 100)

pred = model.predict(dval)

test_pred = model.predict(test_tfidf_vectors)
submit["xgb"] = test_pred

print(classification_report(val_y, pred, target_names=['negative','positive']))